# 使用MMRazor对ResNet34进行剪枝

## 回顾MMCls

In [26]:
# Prepare config path
MMCLS_PATH='/home/liukai/Documents/mmlab2/others/mmclassification/'
config_file=MMCLS_PATH+'configs/resnet/resnet34_8xb32_in1k.py'

In [27]:
# Run config
# !python ./tools/train.py $config_file

## 准备剪枝config

1. 跨库调用resnet34配置文件
2. 增加pretrained参数
3. 将resnet34模型装入剪枝算法wrapper中
4. 配置剪枝比例
5. 运行

In [28]:
from mmengine import Config
prune_config_path='./prune_resnet34.py'
def write_config(config_str,filename):
    with open(filename,'w') as f:
        f.write(config_str)

### 1. 跨库调用resnet34配置文件

首先我们先跨库调用resnet34的配置文件。通过跨库调用，我们可以继承原有配置文件的所有内容。

In [29]:
config_string = """
_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']
"""
write_config(config_string, prune_config_path)
print(Config.fromfile(prune_config_path)['model'])

{'type': 'ImageClassifier', 'backbone': {'type': 'ResNet', 'depth': 34, 'num_stages': 4, 'out_indices': (3,), 'style': 'pytorch'}, 'neck': {'type': 'GlobalAveragePooling'}, 'head': {'type': 'LinearClsHead', 'num_classes': 1000, 'in_channels': 512, 'loss': {'type': 'CrossEntropyLoss', 'loss_weight': 1.0}, 'topk': (1, 5)}, '_scope_': 'mmcls'}


/home/liukai/Documents/mmlab2/others/max/mmengine/mmengine/config/utils.py:50: UserWarning: There is not `Config` define in {'Name': 'convnext-base_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 15359124480, 'Parameters': 88591464}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.openmmlab.com/mmclassification/v0/convnext/convnext-base_3rdparty_in21k_20220124-13b83eec.pth', 'Converted From': {'Weights': 'https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth', 'Code': 'https://github.com/facebookresearch/ConvNeXt'}}
  warnings.warn(f'There is not `Config` define in {model_cfg}')
/home/liukai/Documents/mmlab2/others/max/mmengine/mmengine/config/utils.py:50: UserWarning: There is not `Config` define in {'Name': 'convnext-large_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 34368026112, 'Parameters': 197767336}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.openmmlab.com/mm

### 2. 增加预训练参数
我们将原有的’model‘字段取出，命名为architecture，并且给archtecture增加init_cfg字段用来加载预训练模型参数。

In [30]:
config_string += """\n
data_preprocessor = {'type': 'mmcls.ClsDataPreprocessor'}
architecture = _base_.model
architecture.update({
    'init_cfg': {
        'type':
        'Pretrained',
        'checkpoint':
        'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'  # noqa
    }
})
"""
write_config(config_string, prune_config_path)

### 3. 将resnet34模型装入剪枝算法wrapper中

我们将原有的model作为architecture放入到ItePruneAlgorithm算法中，并且将ItePruneAlgorithm作为新的model字段。

In [31]:
config_string+="""
target_pruning_ratio={}
model = dict(
    _delete_=True,
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=architecture,
    mutator_cfg=dict(
        type='ChannelMutator',
        channel_unit_cfg=dict(
            type='L1MutableChannelUnit',
            default_args=dict(choice_mode='ratio'))),
    target_pruning_ratio=target_pruning_ratio,
    step_epoch=1,
    prune_times=1,
)
"""
write_config(config_string, prune_config_path)

配置到这一步时，我们的config文件已经能够运行了。但是因为我们没有配置target_pruning_ratio，因此现在跑起来就和直接用原有config跑起来没有区别，接下来我们会介绍如何配置剪枝比例

In [32]:
#! python ./tools/train.py $prune_config_path

### 4. 配置剪枝比例

我们的模型使用tracer解析模型，进而获得剪枝节点，为了方便用户配置剪枝节点比例，我们提供了一个获得剪枝节点剪枝比例配置的工具。通过该工具，我们可以方便地对剪枝比例进行配置。

In [33]:
! python ./tools/get_channel_units.py $prune_config_path --choice -o prune_ratio_templace.json &> /dev/null 2>&1
! cat prune_ratio_templace.json
! rm prune_ratio_templace.json

{
    "backbone.conv1_(0, 64)_64":1.0,
    "backbone.layer1.0.conv1_(0, 64)_64":1.0,
    "backbone.layer1.1.conv1_(0, 64)_64":1.0,
    "backbone.layer1.2.conv1_(0, 64)_64":1.0,
    "backbone.layer2.0.conv1_(0, 128)_128":1.0,
    "backbone.layer2.0.conv2_(0, 128)_128":1.0,
    "backbone.layer2.1.conv1_(0, 128)_128":1.0,
    "backbone.layer2.2.conv1_(0, 128)_128":1.0,
    "backbone.layer2.3.conv1_(0, 128)_128":1.0,
    "backbone.layer3.0.conv1_(0, 256)_256":1.0,
    "backbone.layer3.0.conv2_(0, 256)_256":1.0,
    "backbone.layer3.1.conv1_(0, 256)_256":1.0,
    "backbone.layer3.2.conv1_(0, 256)_256":1.0,
    "backbone.layer3.3.conv1_(0, 256)_256":1.0,
    "backbone.layer3.4.conv1_(0, 256)_256":1.0,
    "backbone.layer3.5.conv1_(0, 256)_256":1.0,
    "backbone.layer4.0.conv1_(0, 512)_512":1.0,
    "backbone.layer4.0.conv2_(0, 512)_512":1.0,
    "backbone.layer4.1.conv1_(0, 512)_512":1.0,
    "backbone.layer4.2.conv1_(0, 512)_512":1.0
}

我们修改该配置模板如下，并且将替换到我们的剪枝配置文件中。

（该配置来源于：Li, Hao, et al. "Pruning filters for efficient convnets." arXiv preprint arXiv:1608.08710 (2016).）

In [34]:
target_config = """
stage_ratio_1 = 0.7
stage_ratio_2 = 0.7
stage_ratio_3 = 0.7
stage_ratio_4 = 1.0

target_pruning_ratio = {
    "backbone.conv1_(0, 64)_64": stage_ratio_1,
    "backbone.layer1.0.conv1_(0, 64)_64": stage_ratio_1,
    "backbone.layer1.1.conv1_(0, 64)_64": stage_ratio_1,
    "backbone.layer1.2.conv1_(0, 64)_64": stage_ratio_1,
    "backbone.layer2.0.conv1_(0, 128)_128": stage_ratio_2,
    "backbone.layer2.0.conv2_(0, 128)_128": stage_ratio_2,
    "backbone.layer2.1.conv1_(0, 128)_128": stage_ratio_2,
    "backbone.layer2.2.conv1_(0, 128)_128": stage_ratio_2,
    "backbone.layer2.3.conv1_(0, 128)_128": stage_ratio_2,
    "backbone.layer3.0.conv1_(0, 256)_256": stage_ratio_3,
    "backbone.layer3.0.conv2_(0, 256)_256": stage_ratio_3,
    "backbone.layer3.1.conv1_(0, 256)_256": stage_ratio_3,
    "backbone.layer3.2.conv1_(0, 256)_256": stage_ratio_3,
    "backbone.layer3.3.conv1_(0, 256)_256": stage_ratio_3,
    "backbone.layer3.4.conv1_(0, 256)_256": stage_ratio_3,
    "backbone.layer3.5.conv1_(0, 256)_256": stage_ratio_3,
    "backbone.layer4.0.conv1_(0, 512)_512": stage_ratio_4,
    "backbone.layer4.0.conv2_(0, 512)_512": stage_ratio_4,
    "backbone.layer4.1.conv1_(0, 512)_512": stage_ratio_4,
    "backbone.layer4.2.conv1_(0, 512)_512": stage_ratio_4
}
"""

In [35]:
config_string=config_string.replace('target_pruning_ratio={}',target_config)
write_config(config_string,prune_config_path)

### 5. 运行

In [36]:
#! python ./tools/train.py $prune_config_path

In [37]:
# 清理临时文件
# ! rm prune_ratio_templace.json